In [1]:
import pandas as pd
import numpy as np
import os

In [8]:
direc = "data/ltcountry"
ltall = pd.DataFrame()

for filename in os.listdir(direc):
    if filename.endswith(".csv"):
        file_loc = os.path.join(direc, filename)
        with open(file_loc) as f:
            ltnew = pd.read_csv(file_loc)
            print(filename+"has"+str(ltnew.shape[0])+"rows")
            ltall = pd.concat([ltall,ltnew])

            coldiff = ltall.columns.symmetric_difference(ltnew.columns)
            if not coldiff.empty:
                print("filename", filename, "cols",coldiff,sep='\n')

ltbrazil.csvhas1104rows
ltcameroon.csvhas48rows
ltindia.csvhas1056rows
ltpoland.csvhas120rows
ltjapan.csvhas1800rows
ltmexico.csvhas852rows
ltoman.csvhas288rows
filename
ltoman.csv
cols
Index(['latabs'], dtype='object')


In [ ]:
ltall.groupby(['country']).agg({
    'temperature' : ['min','max','mean','count'],
    'latabs' : ['min','max','mean','count']
})

temperature                          latabs                         
                 min    max       mean count     min     max       mean count
country                                                                      
Brazil         12.20  33.93  25.470196   969   0.051  33.517  13.859272  1104
Cameroon       21.87  36.00  27.227941    34   4.020  10.451   7.789250    48
India           1.95  37.40  26.113477  1044   8.300  34.083  21.104516  1056
Japan          -7.15  30.14  15.495854  1797  24.300  45.417  35.664822  1800
Mexico          7.00  34.45  23.230397   806  14.917  32.400  22.114901   852
Oman           12.10  38.39  27.753463   205     NaN     NaN        NaN     0
Poland         -3.85  23.25  10.462750   120  50.078  54.750  52.498500   120

In [14]:
ltall['latabs'] = np.where(ltall.country == 'Oman', ltall.latitude, ltall.latabs)
ltall.groupby(['country']).agg({
    'latabs' : ['min','max','mean','count']
})

latabs                         
             min     max       mean count
country                                  
Brazil     0.051  33.517  13.859272  1104
Cameroon   4.020  10.451   7.789250    48
India      8.300  34.083  21.104516  1056
Japan     24.300  45.417  35.664822  1800
Mexico    14.917  32.400  22.114901   852
Oman      16.933  26.167  21.997875   288
Poland    50.078  54.750  52.498500   120

In [34]:
nls97 = pd.read_csv("data/nls97f.csv")
nls97.set_index('personid', inplace=True)
nls97add = pd.read_csv("data/nls97add.csv")

In [35]:
print(nls97.originalid.nunique()==nls97.shape[0])
nls97add.originalid.nunique() ==nls97add.shape[0]

True


True

In [36]:
#contaminate
nls97.sort_values('originalid',inplace=True)
nls97add.sort_values('originalid', inplace=True)
nls97.iloc[0:2, -1] = nls97[0:2].originalid+10000
nls97.originalid.head(2)

personid
135335    10001
999406    10002
Name: originalid, dtype: int64

In [37]:
nls97add.iloc[0:2, 0] = nls97add[0:2].originalid+20000
nls97add.originalid.head(2)

0    20001
1    20002
Name: originalid, dtype: int64

In [45]:
#left join (merge == join)
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how='left')
nlsnew.loc[nlsnew.originalid >9999, ['originalid', 'gender','motherage','parentincome']]

#right join
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how='right')
nlsnew.loc[nlsnew.originalid >9999, ['originalid', 'gender','motherage','parentincome']]

#inner join
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how='inner')
nlsnew.loc[nlsnew.originalid >9999, ['originalid', 'gender','motherage','parentincome']]

#outer join
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how='outer')
nlsnew.loc[nlsnew.originalid >9999, ['originalid', 'gender','motherage','parentincome']]


,originalid,gender,motherage,parentincome
8982,10001,Female,NaN,NaN
8983,10002,Male,NaN,NaN
8984,20001,NaN,26.0,-3.0
8985,20002,NaN,19.0,-4.0


In [72]:
def checkmerge(dfleft, dfright, idvar):
    dfleft['inleft'] = "Y"
    dfright['inright'] = "Y"
    dfboth = pd.merge(dfleft[[idvar,'inleft']], dfright[[idvar,'inright']], on=[idvar], how='outer')
    dfboth.fillna('N', inplace=True)
    print(pd.crosstab(dfboth.inleft, dfboth.inright))
    print(dfboth.loc[(dfboth.inleft=='N') | (dfboth.inright =='N')])

In [47]:
checkmerge(nls97,nls97add, "originalid")

inright  N     Y
inleft          
N        0     2
Y        2  8982


In [48]:
weeksworked = pd.read_csv('data/nls97weeksworked.csv')
colenr = pd.read_csv('data/nls97colenr.csv')

In [54]:
weeksworked.groupby(['originalid','year'])['originalid'].count().shape

(44920,)

In [58]:
merge_weekschool = pd.merge(weeksworked, colenr, on=['originalid','year'], how='inner')

In [59]:
merge_weekschool

,originalid,year,weeksworked,colenr
0,8245,2000,46.0,1. Not enrolled
1,3962,2000,5.0,1. Not enrolled
2,3571,2000,5.0,1. Not enrolled
3,2979,2000,29.0,3. 4-year college
4,8511,2000,53.0,1. Not enrolled
...,...,...,...,...
44915,7217,2004,15.0,3. 4-year college
44916,2,2004,52.0,3. 4-year college
44917,5113,2004,8.0,1. Not enrolled
44918,7815,2004,0.0,1. Not enrolled


In [85]:
countries = pd.read_csv('data/ltcountries.csv')
locations = pd.read_csv('data/ltlocations.csv')

In [73]:
checkmerge(countries,locations,'countryid')

inright  N      Y
inleft           
N        0      1
Y        2  27472
      countryid inleft inright
7363         FO      N       Y
9716         LQ      Y       N
13104        ST      Y       N


In [76]:
countries.loc[countries.countryid.isin(['LQ','ST'])]

,countryid,country,inleft
124,LQ,Palmyra Atoll [United States],Y
195,ST,Saint Lucia,Y


In [77]:
locations.loc[locations.countryid == 'FO']

,locationid,latitude,longitude,stnelev,station,countryid,inright
7363,FOM00006009,61.4,-6.667,102.0,AKRABERG,FO,Y


In [79]:
countries.set_index(['countryid'],inplace=True)
locations.set_index(['countryid'],inplace=True)

In [80]:
countries.index.nunique() == countries.shape[0]

True

In [ ]:
stations = countries.join(locations)
stations[['locationid','latitude','stnelev','country']]

In [87]:
stations = pd.merge(countries, locations, on=['countryid'], how='left')

In [88]:
stations.shape

(27474, 7)

In [94]:
stations.loc[stations.countryid.isin(['LQ','ST'])].isnull().sum()

countryid     0
country       0
locationid    2
latitude      2
longitude     2
stnelev       2
station       2
dtype: int64